In [102]:
import pandas as pd
import database_connect as db_connect
import stock_marker_binance
import numpy as np
from datetime import datetime

In [2]:
CONFIG = db_connect.load_config()

In [3]:
def hours_to_milliseconds(hours):
    return hours * 60 * 60 * 1000

In [4]:
def insert_data_to_db(conn, df):
    with conn.cursor() as cursor:
        for row in df.itertuples():
            query = f"INSERT INTO trading_bot_historydataeth (timestamp, open, high, low, close, volume, \
            close_time, quote_asset_volume, num_trades) \
            VALUES ('{row.timestamp}', {row.open}, {row.high}, {row.low}, {row.close}, {row.volume}, \
            {row.close_time}, {row.quote_asset_volume}, {row.num_trades})"
            cursor.execute(query)
            conn.commit()

### Adding historical data to the database.

In [ ]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [25]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,01/01/2020 00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504
1,01/01/2020 01:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885
2,01/01/2020 02:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046
3,01/01/2020 03:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818
4,01/01/2020 04:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264


In [27]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [28]:
for row in df.itertuples():
    print(tuple(row))
    break

(0, '01/01/2020 00:00', 129.16, 129.19, 128.68, 128.87, 7769.17336, 1577840000000.0, 1000929.742, 2504)


In [29]:
# insert_data_to_db(conn, df) # Commented to avoid inserting data to db

In [32]:
conn.close()

### Update database with fresh data from Binance

In [63]:
# Every hour the script updates the database with the latest data from the API.
# The script is run using a cron job.

# Identifying the last timestamp in the database and using that as the starting point for the API call.

In [64]:
# Identifying the last timestamp in the database and using that as the starting point for the API call.
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [65]:
with conn.cursor() as cursor:
    cursor.execute("SELECT MAX(timestamp) FROM trading_bot_historydataeth")
    result = cursor.fetchone()
    print(result)
conn.close()

(datetime.datetime(2024, 2, 7, 10, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=18000))),)


In [66]:
INTERVAL_START = result[0]

print(INTERVAL_START)

INTERVAL_START = INTERVAL_START.replace(tzinfo=None)
print(INTERVAL_START -  stock_marker_binance.UNIX_TIME_START)
INTERVAL_START = int((INTERVAL_START -  stock_marker_binance.UNIX_TIME_START).total_seconds()) * 1000 + 1
print(INTERVAL_START)




2024-02-07 10:00:00+05:00
19760 days, 10:00:00
1707300000001


In [67]:
url = 'https://api3.binance.com/api/v3/time'

df_time_stop = pd.read_json(url, typ='series')

TIME_STOP = df_time_stop.iloc[0]
print(TIME_STOP)

# TIME_STOP = int ( time.mktime(TIME_STOP.timetuple())*1000)
TIME_STOP = int(TIME_STOP.timestamp()) * 1000
print(TIME_STOP)

2024-02-09 18:48:06.238000
1707504486000


In [68]:
SYMBOL = 'ETHUSDT'
INTERVAL = '1h'
df_total = pd.DataFrame()

In [69]:
while INTERVAL_START < TIME_STOP:
      df=stock_marker_binance.import_stock_data(SYMBOL, INTERVAL, startTime =INTERVAL_START,  limit=984)
      df.rename(
          columns={0:'timestamp', 1:'open', 2:'high', 3:'low', 4:'close',5:'volume', 6:'close_time', 7:'quote_asset_volume',
                8:'num_trades', 9:'taker_buy_base_asset_volume',10:'taker_buy_quote_asset_volume', 11:'unused_field'},
         inplace=True)
      df.drop(columns=['taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'unused_field'], inplace=True)
      if len(df_total)>0:
            df_total = pd.concat([df_total, df], ignore_index=True)
      else:
            df_total = df.copy()
      INTERVAL_START = INTERVAL_START + hours_to_milliseconds(984)
      print(len(df_total))

56


In [70]:
df_total.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2024-02-07 11:00:00,2355.72,2362.56,2354.64,2361.66,9806.4272,1707307199999,2.313498e+07,32982
1,2024-02-07 12:00:00,2361.67,2373.89,2361.00,2373.34,9119.3380,1707310799999,2.159444e+07,28783
2,2024-02-07 13:00:00,2373.34,2376.64,2365.86,2372.83,14295.9304,1707314399999,3.390730e+07,37141
3,2024-02-07 14:00:00,2372.83,2385.45,2362.60,2365.89,22425.9134,1707317999999,5.324919e+07,57929
4,2024-02-07 15:00:00,2365.89,2376.51,2361.57,2369.66,13655.9583,1707321599999,3.233650e+07,47398
5,2024-02-07 16:00:00,2369.66,2400.00,2368.85,2398.24,23474.8528,1707325199999,5.604651e+07,56563
6,2024-02-07 17:00:00,2398.24,2424.34,2396.41,2410.26,46060.3814,1707328799999,1.112563e+08,83152
7,2024-02-07 18:00:00,2410.25,2418.96,2406.26,2418.18,15753.7126,1707332399999,3.800787e+07,40652
8,2024-02-07 19:00:00,2418.18,2425.46,2413.32,2419.96,16137.2512,1707335999999,3.906651e+07,39536
9,2024-02-07 20:00:00,2419.97,2444.44,2419.96,2439.38,32265.2799,1707339599999,7.851546e+07,65278


In [71]:
conn = db_connect.connect(CONFIG)
insert_data_to_db(conn, df_total)
conn.close()

Connected to the PostgreSQL server.


### Recalculate the additional columns

In [112]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [113]:
with conn.cursor() as cursor:
    cursor.execute("SELECT id, timestamp, open,	high,	low,	close,	volume,	close_time, \
                    quote_asset_volume,	num_trades \
                    FROM trading_bot_historydataeth \
                    WHERE hour_sin is null \
                    ORDER BY timestamp DESC \
                    LIMIT 385")
    
    

    result = cursor.fetchall()
conn.close()

In [114]:
df = pd.DataFrame(result, columns=['id', 'timestamp', 'open', 'high', 'low', 
                                   'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades'])

In [115]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year

In [116]:
df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])
print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 
                                                'quote_asset_volume', 'num_trades'
                                                 ])
print("Complete adding RSI")


df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI


c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:273: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  up[up < 0] = 0
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:273: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  up[up < 0] = 0
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:273: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at

In [117]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_192_quote_asset_volume,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades
0,38190,1.000000,6.123234e-17,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.089859,49.859221,71.389892,3.699243,27.179893,46.358171,50.568465,49.547127,50.109445,49.851729
1,38191,0.965926,-2.588190e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.024354,49.954691,3.279849,7.395918,30.689324,45.776484,49.810307,49.289525,49.963152,49.915666
2,38879,0.866025,-5.000000e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.214929,50.052924,67.893374,68.134645,46.048634,49.646220,50.552824,50.354181,50.269552,50.092061
3,38880,0.707107,-7.071068e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.200925,50.101788,73.389622,73.244999,52.088494,50.497962,51.041344,49.882164,50.246861,50.124498
4,38881,0.500000,-8.660254e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.060243,49.864236,65.633652,52.506210,51.607839,49.419907,50.660680,49.553705,50.089724,49.785607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,39631,-0.500000,-8.660254e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,51.605484,50.764413,60.610369,65.450251,59.595558,52.497791,53.100212,52.741998,51.284766,50.657069
753,39632,-0.707107,-7.071068e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,50.589455,50.504011,40.293519,55.196642,53.095138,46.346020,51.371620,50.317666,50.383280,50.369280
754,39633,-0.866025,-5.000000e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,50.181331,50.230337,48.720519,53.449911,52.730552,45.170616,49.285496,49.934885,50.065080,50.231440
755,39634,-0.965926,-2.588190e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,49.911303,50.235723,0.000000,38.816488,51.291283,48.957793,46.783647,50.680708,49.800694,50.226517


In [118]:
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [119]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RSI_192_quote_asset_volume,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades
0,38190,1.000000,6.123234e-17,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.089859,49.859221,71.389892,3.699243,27.179893,46.358171,50.568465,49.547127,50.109445,49.851729
1,38191,0.965926,-2.588190e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.024354,49.954691,3.279849,7.395918,30.689324,45.776484,49.810307,49.289525,49.963152,49.915666
2,38879,0.866025,-5.000000e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.214929,50.052924,67.893374,68.134645,46.048634,49.646220,50.552824,50.354181,50.269552,50.092061
3,38880,0.707107,-7.071068e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.200925,50.101788,73.389622,73.244999,52.088494,50.497962,51.041344,49.882164,50.246861,50.124498
4,38881,0.500000,-8.660254e-01,0.951057,-0.309017,0.500000,0.866025,0.781831,0.623490,2024,...,50.060243,49.864236,65.633652,52.506210,51.607839,49.419907,50.660680,49.553705,50.089724,49.785607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,39631,-0.500000,-8.660254e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,51.605484,50.764413,60.610369,65.450251,59.595558,52.497791,53.100212,52.741998,51.284766,50.657069
753,39632,-0.707107,-7.071068e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,50.589455,50.504011,40.293519,55.196642,53.095138,46.346020,51.371620,50.317666,50.383280,50.369280
754,39633,-0.866025,-5.000000e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,50.181331,50.230337,48.720519,53.449911,52.730552,45.170616,49.285496,49.934885,50.065080,50.231440
755,39634,-0.965926,-2.588190e-01,0.951057,-0.309017,0.866025,0.500000,-0.433884,-0.900969,2024,...,49.911303,50.235723,0.000000,38.816488,51.291283,48.957793,46.783647,50.680708,49.800694,50.226517


In [120]:
cols = list(df_result.columns)

In [121]:
cols.remove('id')

In [122]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [123]:
for index, row in df_result.iterrows():
    update_sql = "UPDATE trading_bot_historydataeth SET "
    for col in cols:
        update_sql += f'"{col}" = {row[col]}, '
    update_sql = update_sql[:-2] # Remove the last comma and space
    update_sql += f' WHERE id = {int(row["id"])}  and "hour_sin" IS NULL'
    with conn.cursor() as cursor:
        cursor.execute(update_sql)
        conn.commit()
conn.close()

In [124]:
conn.close()